# core

> lisette core

In [ ]:
#| default_exp core

In [ ]:
#| export
import litellm, json
from litellm import completion, stream_chunk_builder, get_model_info
from litellm.types.utils import ModelResponseStream,ModelResponse
from litellm.utils import function_to_dict
from toolslm.funccall import mk_ns, call_func
from toolslm.funccall import get_schema
from typing import Optional
from fastcore.all import *

## LiteLLM

Litellm provides an easy wrapper for most big LLM providers.

In [ ]:
ms = ["gemini/gemini-2.5-flash", "claude-sonnet-4-20250514", "openai/gpt-4.1"]

TODO: test mixed content/tool calls message (and mixed images too).

In [ ]:
#| export
@patch
def _repr_markdown_(self: litellm.ModelResponse):
    message = self.choices[0].message
    content = ''
    if message.content: content += message.content
    if message.tool_calls:
        tool_calls = [f"\n\n🔧 {tc.function.name}({tc.function.arguments})\n" for tc in message.tool_calls]
        content += "\n".join(tool_calls)
    if not content: content = str(message)
    details = [
        f"id: `{self.id}`",
        f"model: `{self.model}`",
        f"finish_reason: `{self.choices[0].finish_reason}`"
    ]
    if hasattr(self, 'usage') and self.usage: details.append(f"usage: `{self.usage}`")
    det_str = '\n- '.join(details)
    
    return f"""{content}

<details>

- {det_str}

</details>"""

In [ ]:
msg = [{'role':'user','content':'Hey there!', 'cache_control': {'type': 'ephemeral'}}]

In [ ]:
for m in ms:
    display(f'=== {m} ===')
    display(completion(m,msg))

### Streaming

In [ ]:
#| export
def stream_with_complete(gen, postproc=noop):
    "Extend streaming response chunks with the complete response"
    chunks = []
    for chunk in gen:
        chunks.append(chunk)
        yield chunk
    postproc(chunks)
    return stream_chunk_builder(chunks)

In [ ]:
from fastcore.xtras import SaveReturn

In [ ]:
model = ms[1]

In [ ]:
r = completion(messages=msg, model=model, stream=True)
r2 = SaveReturn(stream_with_complete(r))

In [ ]:
for o in r2:
    cts = o.choices[0].delta.content
    if cts: print(cts, end='')

Hello! Nice to meet you. How are you doing today?

In [ ]:
r2.value

Hello! Nice to meet you. How are you doing today?

<details>

- id: `chatcmpl-7a42920c-6162-470c-b86a-844ffd1914d6`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=16, prompt_tokens=10, total_tokens=26, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, text_tokens=None), prompt_tokens_details=None)`

</details>

### Tools

In [ ]:
#| export
def _lite_mk_func(f):
    if isinstance(f, dict): return f
    return {'type':'function', 'function':get_schema(f, pname='parameters')}

In [ ]:
def simple_add(
    a: int,   # first operand
    b: int=0  # second operand
) -> int:
    "Add two numbers together"
    print(f"TOOL CALLED {a=} + {b=}")
    return a + b

In [ ]:
toolsc = _lite_mk_func(simple_add)
toolsc

{'type': 'function',
 'function': {'name': 'simple_add',
  'description': 'Add two numbers together\n\nReturns:\n- type: integer',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': 'first operand'},
    'b': {'type': 'integer', 'description': 'second operand', 'default': 0}},
   'required': ['a']}}}

In [ ]:
#| export
def mk_user(s, cache=False):
    res = {"role": "user", "content": s}
    if cache: res['cache_control'] = {'type': 'ephemeral'}
    return res

In [ ]:
tmsg = mk_user("What is 5478954793+547982745? How about 5479749754+9875438979? Always use tools for calculations, and describe what you'll do before using a tool. Where multiple tool calls are required, do them in a single response where possible.")
r = completion(model, [tmsg], tools=[toolsc])

In [ ]:
r

I'll help you calculate both of those additions using the simple_add tool. Let me perform both calculations for you:

🔧 simple_add({"a": 5478954793, "b": 547982745})



🔧 simple_add({"a": 5479749754, "b": 9875438979})


<details>

- id: `chatcmpl-d2b6869e-e336-46a9-9162-1f265c9d5104`
- model: `claude-sonnet-4-20250514`
- finish_reason: `tool_calls`
- usage: `Usage(completion_tokens=159, prompt_tokens=475, total_tokens=634, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0)`

</details>

In [ ]:
#| export
def _lite_call_func(tc,ns,raise_on_err=True):
    res = call_func(tc.function.name, json.loads(tc.function.arguments),ns=ns)
    return {"tool_call_id": tc.id, "role": "tool", "name": tc.function.name, "content": str(res)}

In [ ]:
tcs = [_lite_call_func(o, ns=globals()) for o in r.choices[0].message.tool_calls]
tcs

TOOL CALLED a=5478954793 + b=547982745
TOOL CALLED a=5479749754 + b=9875438979


[{'tool_call_id': 'toolu_01ANyXaH1yQ8pTkWKh89pQsQ',
  'role': 'tool',
  'name': 'simple_add',
  'content': '6026937538'},
 {'tool_call_id': 'toolu_01AwCC8kjj42YFhSRLX3Uo6n',
  'role': 'tool',
  'name': 'simple_add',
  'content': '15355188733'}]

In [ ]:
#| export
def delta_text(msg):
    "Extract printable content from streaming delta, return None if nothing to print"
    c = msg.choices[0]
    if not c: return c
    if not hasattr(c,'delta'): return None #f'{c}'
    delta = c.delta
    if delta.content: return delta.content
    if delta.tool_calls:
        res = ''.join(f"🔧 {tc.function.name}" for tc in delta.tool_calls if tc.id and tc.function.name)
        if res: return f'\n{res}'
    if hasattr(delta,'reasoning_content'): return '🧠' if delta.reasoning_content else '\n\n'
    return None

In [ ]:
r = completion(messages=[tmsg], model=model, stream=True, tools=[toolsc])
r2 = SaveReturn(stream_with_complete(r))
for o in r2: print(delta_text(o) or '', end='')

I'll help you calculate both of those addition problems using the simple_add tool

. Let me perform both calculations for you:


🔧 simple_add


🔧 simple_add

In [ ]:
r2.value

I'll help you calculate both of those addition problems using the simple_add tool. Let me perform both calculations for you:

🔧 simple_add({"a": 5478954793, "b": 547982745})



🔧 simple_add({"a": 5479749754, "b": 9875438979})


<details>

- id: `chatcmpl-85cad387-08ec-46dc-99d7-8e6003743730`
- model: `claude-sonnet-4-20250514`
- finish_reason: `tool_calls`
- usage: `Usage(completion_tokens=160, prompt_tokens=475, total_tokens=635, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, text_tokens=None), prompt_tokens_details=None)`

</details>

In [ ]:
msg = mk_user("Solve this complex math problem: What is the derivative of x^3 + 2x^2 - 5x + 1?")
r = completion(messages=[msg], model=model, stream=True, reasoning_effort="low")
r2 = SaveReturn(stream_with_complete(r))
for o in r2: print(delta_text(o) or '', end='')


🧠

🧠🧠

🧠🧠🧠🧠

🧠🧠

🧠

...

 d/dx(-5x) = -5 × 1x⁰ = -5

4) d/dx(1) =

 0 (derivative of a constant is zero)

**Therefore

:**
f'(x) = 3x² + 4x - 5

The derivative of x³ + 2x² - 

5x + 1 is **3x² + 4x -

 5**.

In [ ]:
r2.value

I'll find the derivative of f(x) = x³ + 2x² - 5x + 1 using the power rule.

**Step-by-step solution:**

Using the power rule: if f(x) = xⁿ, then f'(x) = nxⁿ⁻¹

Taking the derivative of each term:

1) d/dx(x³) = 3x²

2) d/dx(2x²) = 2 × 2x¹ = 4x

3) d/dx(-5x) = -5 × 1x⁰ = -5

4) d/dx(1) = 0 (derivative of a constant is zero)

**Therefore:**
f'(x) = 3x² + 4x - 5

The derivative of x³ + 2x² - 5x + 1 is **3x² + 4x - 5**.

<details>

- id: `chatcmpl-7f7397d8-b6c3-4916-87b8-2245024988cf`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=412, prompt_tokens=66, total_tokens=478, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=158, rejected_prediction_tokens=None, text_tokens=None), prompt_tokens_details=None)`

</details>

In [ ]:
search_tool = { "type": "web_search_20250305", "name": "web_search", "max_uses": 3}
smsg = mk_user("Search the web and tell me very briefly about otters")
r = completion(ms[1], [smsg], tools=[search_tool])
r

Otters are carnivorous mammals in the weasel family, found on every continent except Australia and Antarctica. There are 13-14 species ranging from small-clawed otters to giant otters, with sizes from 0.6 to 1.8 meters in length.

They have elongated bodies, long tails, webbed feet for swimming, and dense waterproof fur. Otters have the densest fur of any animal—as many as a million hairs per square inch. Their lung capacity is 2.5 times greater than similar-sized land mammals, allowing sea otters to stay underwater for over 5 minutes and river otters up to 8 minutes.

All otters are expert hunters that eat fish, crustaceans, and other critters. Sea otters famously use tools, floating on their backs and smashing mollusks on rocks placed on their chests. They are notably playful animals, engaging in sliding and water games.

Baby otters stay with their mothers for up to a year, and otters can live up to 16 years. Unfortunately, many species remain at risk from pollution, habitat loss, and historical hunting for their fur.

<details>

- id: `chatcmpl-ff33e062-ce19-4e53-a0bb-1677026bbf9b`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=532, prompt_tokens=13316, total_tokens=13848, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), server_tool_use=ServerToolUse(web_search_requests=1), cache_creation_input_tokens=0, cache_read_input_tokens=0)`

</details>

When not using streaming, all citations are placed in a separate key in the response:

In [ ]:
r.choices[0].message.provider_specific_fields['citations'][0]

[{'type': 'web_search_result_location',
  'cited_text': 'The charismatic otter, a member of the weasel family, is found on every continent except Australia and Antarctica. ',
  'url': 'https://www.nationalgeographic.com/animals/mammals/facts/otters-1',
  'title': 'Otters, facts and information | National Geographic',
  'encrypted_index': 'Eo8BCioIBxgCIiQ4ODk4YTFkYy0yMTNkLTRhNmYtOTljYi03ZTBlNTUzZDc0NWISDGSBf88S47DAweIwGhoMNZkbIflRzVJKEINOIjD/KVAIb0wABKB3JbDuVzoI+duKRiEWJxUFPg34RfUtoocpXjbS0n/A4z5bSsGJGX0qE4dOrL4dvQN149N3Db7KxKbXF3kYBA=='}]

In [ ]:
r = list(completion(ms[1], [smsg], tools=[search_tool], stream=True))

In [ ]:
#| export
def cite_footnotes(stream_list):
    "Add markdown footnote citations to stream deltas"
    for msg in stream_list:
        delta = nested_idx(msg, 'choices', 0, 'delta')
        if not delta: continue
        citation = nested_idx(delta, 'provider_specific_fields', 'citation')
        if citation:
            title = citation['title'].replace('"', '\\"')
            delta.content = f'[*]({citation["url"]} "{title}") '

In [ ]:
cite_footnotes(r)
stream_chunk_builder(r)

Otters are [*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") carnivorous mammals in the weasel family, found on every continent except Australia and Antarctica. [*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") [*](https://en.wikipedia.org/wiki/Otter "Otter - Wikipedia") There are 13-14 species in total, ranging from the small-clawed otter to the giant otter.

[*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") Most are small, with short ears and noses, elongated bodies, long tails, and soft, dense fur. [*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") Otters have the densest fur of any animal—as many as a million hairs per square inch in places. [*](https://en.wikipedia.org/wiki/Otter "Otter - Wikipedia") They have long, slim bodies with relatively short limbs and powerful webbed feet used for swimming.

[*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") All otters are expert hunters that eat fish, crustaceans, and other critters. [*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") Sea otters use an ingenious method to open shellfish—floating on their backs, placing a rock on their chest, then smashing mollusks against it. [*](https://www.doi.gov/blog/12-facts-about-otters-sea-otter-awareness-week "12 Facts About Otters for Sea Otter Awareness Week | U.S. Department of the Interior") An otter's lung capacity is 2.5 times greater than similar-sized land mammals, with sea otters staying submerged for more than 5 minutes and river otters for up to 8 minutes.

[*](https://en.wikipedia.org/wiki/Otter "Otter - Wikipedia") They are playful animals, engaging in activities like sliding into water on natural slides and playing with stones. [*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") When sea otters nap, they entangle themselves in kelp so they don't float away, and sometimes intertwine their feet with another sea otter to stay together.

[*](https://www.nationalgeographic.com/animals/mammals/facts/otters-1 "Otters, facts and information | National Geographic") Otters were once hunted extensively for their fur, many to the point of near extinction, and despite regulations, many species remain at risk from pollution and habitat loss.

<details>

- id: `chatcmpl-47e5ae35-2e10-4663-9416-c52f991acb21`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=584, prompt_tokens=13316, total_tokens=13900, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=0, rejected_prediction_tokens=None, text_tokens=None), prompt_tokens_details=None)`

</details>

## Chat

Litellm is pretty bare bones. It doesnt keep track of conversation history or anything.

So lets make a claudette style wrapper so we can do streaming, toolcalling, and toolloops without problems.

In [ ]:
#| export
# TODO: dont like this var name...
# TODO: make enum so type hints are nice
effort = AttrDict({o[0]:o for o in ('low','medium','high')})

In [ ]:
#| export
class Chat:
    def __init__(self, model:str, sp='', temp=0, tools:list=None, hist:list=None, ns:Optional[dict]=None, cache=False):
        "LiteLLM chat client."
        self.model = model
        hist,tools = listify(hist),listify(tools)
        if ns is None and tools: ns = mk_ns(tools)
        elif ns is None: ns = globals()
        self.tool_schemas = [_lite_mk_func(t) for t in tools] if tools else None
        store_attr()
    
    def _prepare_msgs(self, msg=None, prefill=None):
        "Prepare the messages list for the API call"
        msgs = [{"role": "system", "content": self.sp}] if self.sp else []
        self.hist += [mk_user(msg, cache=self.cache)] if isinstance(msg, str) \
            else [msg] if isinstance(msg, dict) \
            else [] if msg is None \
            else msg
        if prefill and get_model_info(self.model)["supports_assistant_prefill"]: 
            self.hist.append({"role":"assistant","content":prefill})
        return msgs + [m if isinstance(m, dict) else m.model_dump() for m in self.hist]

    def _call(self, msg=None, prefill=None, temp=None, think=None, stream=False, max_tool_rounds=1, tool_round=0, final_prompt=None, tool_choice=None, **kwargs):
        "Internal method that always yields responses"
        msgs = self._prepare_msgs(msg, prefill)
        res = completion(model=self.model, messages=msgs, stream=stream, 
                         tools=self.tool_schemas, reasoning_effort = effort.get(think),
                         # temperature is not supported when reasoning
                         temperature=None if think else (temp if temp is not None else self.temp), **kwargs)
        if stream: res = yield from stream_with_complete(res, postproc=cite_footnotes)
        m = res.choices[0].message
        self.hist.append(m)
        yield res

        if tcs := m.tool_calls:
            tool_results = [_lite_call_func(tc, ns=self.ns) for tc in tcs]
            if tool_round>=max_tool_rounds-1:
                tool_results += ([{"role": "user", "content": final_prompt}] if final_prompt else [])
                tool_choice='none'
            yield from self._call(
                tool_results, stream, max_tool_rounds, tool_round+1,
                final_prompt, tool_choice=tool_choice, **kwargs)
    
    def __call__(self, msg=None, prefill=None, temp=None, think=None, stream=False, max_tool_rounds=1,
                 final_prompt=None, return_all=False, **kwargs):
        "Main call method - handles streaming vs non-streaming"
        result_gen = self._call(msg, prefill, temp, think, stream, max_tool_rounds, 0, final_prompt, **kwargs)     
        if stream: return result_gen              # streaming
        elif return_all: return list(result_gen)  # toolloop behavior
        else: return last(result_gen)             # normal chat behavior

## Add prefill

Litellm supports `prefill` for models that have this feature. Note, it does not add your prefill to the response, so you'll have to do that yourself in post-processing.

In [ ]:
chat = Chat(ms[1])
chat("Hey my name is Rens", prefill="Howdy Re")

-ns! Nice to meet you. How are you doing today?

<details>

- id: `chatcmpl-2519bf61-03e9-4487-adee-63560158af1e`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=17, prompt_tokens=18, total_tokens=35, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0)`

</details>

### Test history tracking

In [ ]:
chat = Chat(m)
res = chat("Hey my name is Rens")
res

Hi Rens! Nice to meet you. How can I help you today? 😊

<details>

- id: `chatcmpl-CDlVYCRj8tekPIc0YXDdq0312VGFt`
- model: `gpt-4.1-2025-04-14`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=17, prompt_tokens=13, total_tokens=30, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None))`

</details>

In [ ]:
chat("Whats my name")

Your name is Rens!

<details>

- id: `chatcmpl-CDlVZiSvndtoiiivjaMR7EaT0FoCp`
- model: `gpt-4.1-2025-04-14`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=6, prompt_tokens=41, total_tokens=47, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None))`

</details>

See now we keep track of history!

### Testing streaming

In [ ]:
from time import sleep
chat2 = Chat(m)
stream_gen = chat2("Count to 5", stream=True)
for chunk in stream_gen:
    sleep(0.1)  # for effect
    if isinstance(chunk, ModelResponse): display(chunk)
    else: print(delta_text(chunk) or '',end='')

## Test tool use

Ok now lets test tool use

In [ ]:
for m in ms:
    display(f'=== {m} ===')
    chat = Chat(m, tools=[simple_add])
    res = chat("What's 5 + 3?")
    display(res)

In [ ]:
chat = Chat(ms[1], tools=[search_tool])
res = chat("Search the web and tell me very briefly about otters", stream=True)
for o in res:
    if isinstance(o, ModelResponse): sleep(0.01); display(o)
    else: print(delta_text(o) or '',end='')

## Test multi tool calling

In [ ]:
chat = Chat(model, tools=[simple_add])
res = chat("What's ((5 + 3)+7)+11? Work step by step", return_all=True, max_tool_rounds=5)
for r in res: display(r)

In [ ]:
@patch(as_prop=True)
def cost(self: Chat):
    "Total cost of all responses in conversation history"
    return sum(getattr(r, '_hidden_params', {}).get('response_cost')  or 0
               for r in self.h if hasattr(r, 'choices'))

Some models support parallel tool calling. I.e. sending multiple tool call requests in one conversation step.

In [ ]:
def multiply(a: int, b: int) -> int:
    "Multiply two numbers"
    print(f"MULTIPLY: {a} * {b}")
    return a * b

chat = Chat(ms[-1], tools=[simple_add, multiply])
res = chat("Calculate (5 + 3) * (7 + 2)", max_tool_rounds=5, return_all=True)
for r in res: display(r)

See it did the additions in one go!

Hit max_tool_rounds limit with final_prompt

In [ ]:
def divide(a: int, b: int) -> float:
    "Divide two numbers"
    display(f"DIVIDE: {a} / {b}")
    return a / b

chat = Chat(m, tools=[simple_add, multiply, divide])
res = chat("Calculate ((10 + 5) * 3) / (2 + 1) step by step", 
           max_tool_rounds=2, return_all=True,
           final_prompt="Please summarize what you've calculated so far")
print(f"Got {len(res)} responses")
for r in res: display(r)

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
from IPython.display import Image

In [ ]:
fn = Path('samples/puppy.jpg')
Image(filename=fn, width=200)

In [ ]:
def _mk_img(data:bytes)->tuple:
    "Convert image bytes to a base64 encoded image"
    img = base64.b64encode(data).decode("utf-8")
    mtype = mimetypes.types_map["."+imghdr.what(None, h=data)]
    return img, mtype

In [ ]:
import base64
import mimetypes
from fastcore import imghdr

In [ ]:
imgbytes = fn.read_bytes()
img,mtype = _mk_img(imgbytes)
imgd = { "image_url": {"url": f'data:{mtype};base64,{img}', "format":mtype} }

In [ ]:
response = completion( model=model, 
    messages=[
        { "role": "user",
        "content": [{ "type": "text", "text": "What’s in this image?" },
        { "type": "image_url", **imgd }] }
    ])

In [ ]:
response